<a href="https://colab.research.google.com/github/tam8738/KHDL/blob/main/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/KHDL/data/processed_data2_cleaned.csv')

In [9]:
# Kiểm tra cấu trúc
print("Cấu trúc dữ liệu:")
print(data.info())

Cấu trúc dữ liệu:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5130 entries, 0 to 5129
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Body          5130 non-null   object
 1   Label         5130 non-null   int64 
 2   Cleaned_Body  5130 non-null   object
dtypes: int64(1), object(2)
memory usage: 120.4+ KB
None


In [10]:
# Kiểm tra phân bố nhãn
print("\nPhân bố nhãn:")
print(data['Label'].value_counts())


Phân bố nhãn:
Label
0    3880
1    1250
Name: count, dtype: int64


In [11]:
# Kiểm tra giá trị thiếu
print("\nGiá trị thiếu:")
print(data.isnull().sum())


Giá trị thiếu:
Body            0
Label           0
Cleaned_Body    0
dtype: int64


In [12]:
#Chuyển đổi văn bản thành đặc trưng số
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X = vectorizer.fit_transform(data['Cleaned_Body'])
y = data['Label']

In [13]:
#xử lý mất cân bằng lớp
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', random_state=42)

In [14]:
#chia dữ liệu: Chia thành tập huấn luyện (80%) và kiểm tra (20%).
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
#khởi tạo và huấn luyện
model = LogisticRegression(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)

LogisticRegression(class_weight='balanced', random_state=42)

In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5, scoring='f1')
print("Cross-validation F1 scores:", scores)
print("Average CV F1 score:", scores.mean())

Cross-validation F1 scores: [0.95607235 0.94147583 0.94117647 0.95336788 0.96143959]
Average CV F1 score: 0.9507064226636096


In [17]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       762
           1       0.94      0.94      0.94       264

    accuracy                           0.97      1026
   macro avg       0.96      0.96      0.96      1026
weighted avg       0.97      0.97      0.97      1026

Confusion Matrix:
[[747  15]
 [ 15 249]]


In [19]:
import joblib

joblib.dump(model, '/content/drive/MyDrive/KHDL/spam_classifier_model.pkl')
joblib.dump(vectorizer, '/content/drive/MyDrive/KHDL/tfidf_vectorizer.pkl')

['/content/drive/MyDrive/KHDL/tfidf_vectorizer.pkl']